# Paris Ratchet - Decomposition
This notebook uses the asymmetric decomposition method from Silicone to split an aggregate greenhouse gas emissions trajectory into CO2, CH4, N2O and F-Gases

## Importing data and tools
This part collects and cleans the data.

In [1]:
import os.path

import pandas as pd
import pyam
import matplotlib.pyplot as plt
import numpy as np
import warnings

from silicone.utils import (
    download_or_load_sr15,
    convert_units_to_MtCO2_equiv,
    return_cases_which_consistently_split,
)
import silicone.multiple_infillers as mi
import silicone.database_crunchers as cr

<IPython.core.display.Javascript object>

In [2]:
#Load SR1.5 database against which to infill
input_complete_data = "./Silicone/silicone_inputs/complete_sr15.csv"
df = download_or_load_sr15(input_complete_data)
# We have to make the columns consistent

df = pyam.IamDataFrame(df.data)

pyam - INFO: Running in a notebook, setting up a basic logging config at level INFO
pyam.core - INFO: Reading file Silicone\silicone_inputs\complete_sr15.csv
numexpr.utils - INFO: NumExpr defaulting to 8 threads.


### Load future emissions based on different ratchet interpretations

**This script runs a single decomposition analysis. Each time you run it with a different input file.

In [25]:
#Different ratchet definitions (see Main Text for details)
#input_file = "./Silicone/silicone_inputs/ratcheted_emissions_gr.csv"
#input_file = "./Silicone/silicone_inputs/ratcheted_emissions_gr_abs.csv"
input_file = "./Silicone/silicone_inputs/ratcheted_emissions_abs.csv"


We have to format the CVS file to make a pyam dataframe.

In [26]:
aggregated_ghg = pd.read_csv(input_file, header=[0, 1], index_col=0)
aggregated_ghg = aggregated_ghg.transpose().reset_index()

In [27]:
aggregated_ghg = aggregated_ghg.rename(columns={"level_0": "model", "level_1": "scenario"})

In [28]:
aggregate = "Emissions|Kyoto Gases (AR5-GWP100)"
aggregated_ghg["variable"] = aggregate
aggregated_ghg["unit"] = "Mt CO2-equiv/yr" # We will convert units later
aggregated_ghg["region"] = "World"

In [29]:
# Convert from GtCO2eq to MtCO2eq
aggregated_ghg.iloc[:,2:-3] = aggregated_ghg.iloc[:,2:-3]*1000
aggregated_ghg = pyam.IamDataFrame(aggregated_ghg).filter(year=range(2010, 2020), keep=False)
aggregated_ghg.head()

,model,scenario,region,variable,unit,year,value
0,paris_ratchet,2ratchet_ndc,World,Emissions|Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2020.0,50639.141460
1,paris_ratchet,2ratchet_ndc,World,Emissions|Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2021.0,52587.959046
2,paris_ratchet,2ratchet_ndc,World,Emissions|Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2022.0,52751.295110
3,paris_ratchet,2ratchet_ndc,World,Emissions|Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2023.0,52914.631174
4,paris_ratchet,2ratchet_ndc,World,Emissions|Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2024.0,53077.967238


Now we can set up the variables to decompose the aggregate into its components. 

In [30]:
components = [
    "Emissions|CO2",
    "Emissions|CH4",
    "Emissions|N2O",
    "Emissions|F-Gases",
]
unit_consistent_db = convert_units_to_MtCO2_equiv(
    df.filter(variable=components)
)
unit_consistent_db.variables(True)

,variable,unit
0,Emissions|CH4,Mt CO2-equiv/yr
1,Emissions|CO2,Mt CO2/yr
2,Emissions|F-Gases,Mt CO2-equiv/yr
3,Emissions|N2O,Mt CO2-equiv/yr


We interpolate to ensure that each model is represented in each year, to prevent variation in which models feature in particular years

In [31]:
for year in unit_consistent_db.filter(variable="Emissions|F-Gases")["year"].unique():
    year = int(year) # Hopefully temporary bug connected to how interpolate works in pyam
    unit_consistent_db.interpolate(year)

<ipython-input-31-d96011fa135e>:3: DeprecationWarning: This method is deprecated and will be removed in future versions. Behavior of `interpolate` will change to `inplace=False` as default in a future release. Set the kwarg explicitly to avoid this warning. Use `inplace=True` to keep current behavior.
  unit_consistent_db.interpolate(year)


## Asymmetric approach: treat the carbon separately
The problem with using the other infillers is that they are not guaranteed to preserve the total. The `split_collection_with_remainder_emissions` function will ensure that this happens, with one of the emissions making up the difference. Typically this will be carbon, as it has the largest amount of emissions and also the results can theoretically go negative. The function can use any of the crunchers, although we will use the quantile rolling windows method. 

In [32]:
non_co2_compon = [
    "Emissions|CH4",
    "Emissions|N2O",
    "Emissions|F-Gases",
]
co2 = "Emissions|CO2"
split_remainder = mi.SplitCollectionWithRemainderEmissions(df)
asym_infilled = split_remainder.infill_components(
    aggregate, non_co2_compon, co2, aggregated_ghg.filter(year=np.arange(2020, 2101, 10)), cr.QuantileRollingWindows
)

In [33]:
asym_infilled.head()

,model,scenario,region,variable,unit,year,value
0,paris_ratchet,2ratchet_ndc,World,Emissions|CH4,Mt CH4/yr,2020.0,335.627945
1,paris_ratchet,2ratchet_ndc,World,Emissions|CH4,Mt CH4/yr,2030.0,311.941044
2,paris_ratchet,2ratchet_ndc,World,Emissions|CH4,Mt CH4/yr,2040.0,212.942804
3,paris_ratchet,2ratchet_ndc,World,Emissions|CH4,Mt CH4/yr,2050.0,189.784891
4,paris_ratchet,2ratchet_ndc,World,Emissions|CH4,Mt CH4/yr,2060.0,178.296505


In [34]:
asym_infilled.variables(True)

if input_file == "./Silicone/silicone_inputs/ratcheted_emissions_gr.csv":
    asym_infilled.to_csv('./Silicone/silicone_outputs/decomposed_emissions_gr.csv',iamc_index=False)
    
if input_file == "./Silicone/silicone_inputs/ratcheted_emissions_gr_abs.csv":
    asym_infilled.to_csv('./Silicone/silicone_outputs/decomposed_emissions_gr_abs.csv',iamc_index=False)

if input_file == "./Silicone/silicone_inputs/ratcheted_emissions_abs.csv":
    asym_infilled.to_csv('./Silicone/silicone_outputs/decomposed_emissions_abs.csv',iamc_index=False)

